In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/Kannada-MNIST/train.csv
/kaggle/input/Kannada-MNIST/sample_submission.csv
/kaggle/input/Kannada-MNIST/test.csv
/kaggle/input/Kannada-MNIST/Dig-MNIST.csv


In [2]:
train = pd.read_csv('../input/Kannada-MNIST/train.csv')
test = pd.read_csv('../input/Kannada-MNIST/test.csv')
sample_submission = pd.read_csv('../input/Kannada-MNIST/sample_submission.csv')
col = ['pixel%d'%i for i in range(784)]

In [3]:
X_train, X_val, Y_train, Y_val = train_test_split(train[col], train['label'], test_size=0.1)
dtrain = xgb.DMatrix(X_train, label=Y_train)
dval = xgb.DMatrix(X_val, label=Y_val)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [4]:
watchlist = [(dval, 'eval'), (dtrain, 'train')]

xgb_params = {
    "objective" : "multi:softmax",
    "eval_metric" : "mlogloss",
    "num_class" : 10,
    "max_depth" : 12,
    "eta" : 0.05,
    "subsample" : 0.9,
    "colsample_bytree" : 0.9,
}

xgb_clf = xgb.train(xgb_params, dtrain, 4000, 
                    watchlist, 
                    early_stopping_rounds=20, 
                    verbose_eval=20)

[0]	eval-mlogloss:2.09604	train-mlogloss:2.08892
Multiple eval metrics have been passed: 'train-mlogloss' will be used for early stopping.

Will train until train-mlogloss hasn't improved in 20 rounds.
[20]	eval-mlogloss:0.711389	train-mlogloss:0.650586
[40]	eval-mlogloss:0.333997	train-mlogloss:0.266429
[60]	eval-mlogloss:0.182456	train-mlogloss:0.117609
[80]	eval-mlogloss:0.115579	train-mlogloss:0.055469
[100]	eval-mlogloss:0.083826	train-mlogloss:0.02809
[120]	eval-mlogloss:0.067319	train-mlogloss:0.015591
[140]	eval-mlogloss:0.05823	train-mlogloss:0.009563
[160]	eval-mlogloss:0.052551	train-mlogloss:0.006375
[180]	eval-mlogloss:0.049167	train-mlogloss:0.004564
[200]	eval-mlogloss:0.046897	train-mlogloss:0.003467
[220]	eval-mlogloss:0.045312	train-mlogloss:0.002761
[240]	eval-mlogloss:0.04423	train-mlogloss:0.002269
[260]	eval-mlogloss:0.043331	train-mlogloss:0.001922
[280]	eval-mlogloss:0.042753	train-mlogloss:0.001665
[300]	eval-mlogloss:0.04214	train-mlogloss:0.001467
[320]	eval-

In [5]:
res = xgb_clf.predict( xgb.DMatrix(test[col]) ).astype(int)

In [6]:
sample_submission['label'] = res
sample_submission.to_csv('submission.csv', index=False)